# Census block synthetic people using GPU

In this notebook, we want to create a dataset of synthetic people based on Census block population data. Census blocks are provided as non-overlapping polygons. We'll represent each synthetic person as a point of lat-lon coordinates. Points will be first generated randomly using normal distribution and then will be checked if they belong to some polygons. To run this at scale, we'll utilize the GPU by using cupy for randomizing the point data, and cuspatial for checking points in polygons.

In [ ]:
import geopandas as gpd
import cudf, cupy, cuspatial

# disbale warning related to quadtree scale setting
import warnings
warnings.filterwarnings("ignore")


## Load data

We'll load Census population data from parquet, and create an unique integer geoid for each polygon.

In [ ]:
gdf = gpd.read_parquet('census-parquet/part.6.parquet')

geoids = cupy.array(range(len(gdf)))


As our polygons do not overlap, a point can only belong to at most 1 polygon. In addition, points data can have different density over different locations. We'll use quadtree data structure to store our points data. Let's read the data in a proper format so that cuspatial can understand and help use build quadtrees easily.

In [ ]:
poly_offsets, ring_offsets, coords = cuspatial.read_polygon_shapefile('part6.shp')


Extract xcoords and ycoords from input data.

In [ ]:
xcoords = coords.x
ycoords = coords.y


## Polygons to points

Below is an util function to create points in polygons. We'll do it in a vectorize maner so that we can run multiple polygons at once.

In [ ]:
def polygons_to_points(
    gdf: gpd.GeoDataFrame,            # table of all polygons
    val_col: str,                     # name of population column in the table
    poly_offsets: cudf.Series,        # polygon offsets
    ring_offsets: cudf.Series,        # polygon offsets
    xcoords: cudf.Series,             # flatten xcoords of polygons
    ycoords: cudf.Series,             # flatten ycoords of polygons
    geoids: cupy.ndarray,             # indexes of polygons
    max_depth: float,                 # max depth of a quadtree
    min_size: float,                  # minimum number of points for a non-leaf quadtree node
    max_len: int,                     # max number of points generated in an iteration
):
    # total number of points to generate
    total_points = gdf[val_col].sum()
    # number of points in each polygon
    num_points = cupy.asarray(gdf[val_col])    
    # polygons with missing points
    keep_polygons = cupy.arange(len(gdf))
    num_polygons = gdf.shape[0]
    # number of points generated in each polygon
    gen_points = cupy.zeros(num_polygons)
    # cudf DataFrame to store all points that is within an arbitrary polygon 
    keep_points = cudf.DataFrame({'x': [], 'y': [], 'geoid': []})
    # bounding box of each polygon
    polygon_bboxes = cuspatial.polygon_bounding_boxes(
        poly_offsets, ring_offsets, xcoords, ycoords
    )
    # num iterations
    it = 0
    while keep_polygons.shape[0] > 0:
        it += 1
        print('it', it, len(keep_polygons))                
        # bounds of polygons
        x_min, y_min, x_max, y_max = gdf.iloc[keep_polygons.get()].total_bounds
        # random points
        xs = cupy.random.uniform(x_min, x_max, max_len)
        ys = cupy.random.uniform(y_min, y_max, max_len)
        # bounds of all the points we have just generated
        min_px, max_px, min_py, max_py = xs.min(), xs.max(), ys.min(), ys.max()
        # calculate scale of the quadtree
        scale = max(max_px - min_px, max_py - min_py) // (1 << max_depth)
        # build a quadtree
        key_to_point, quadtree = cuspatial.quadtree_on_points(
            xs, ys, min_px, max_px, min_py, max_py, scale, max_depth, min_size
        )
        poly_quad_pairs = cuspatial.join_quadtree_and_bounding_boxes(
            quadtree, polygon_bboxes, x_min, x_max, y_min, y_max, scale, max_depth
        )
        result = cuspatial.quadtree_point_in_polygon(
            poly_quad_pairs, quadtree, key_to_point, xs, ys,
            poly_offsets, ring_offsets, xcoords, ycoords
        )
        # filtering result to only keep points of polygons that not completely full
        result_array = cupy.asarray(result.polygon_index)
        keep_ids = cupy.isin(result_array, keep_polygons)
        keep_ids = cupy.where(keep_ids > 0)[0]
        # keep_ids = cupy.isin(result_array, keep_polygons)
        # keep_ids = result_array[cupy.where(keep_ids==1)[0]]
        result = result.iloc[keep_ids]
        keep_points = cudf.concat([
            keep_points,
            cudf.DataFrame({
                'x': xs[result['point_index']],
                'y': ys[result['point_index']],
                'geoid': geoids[result['polygon_index']]})
            ], ignore_index=True)
        count_df = result.groupby('polygon_index').count().reset_index()
        gen_points[count_df['polygon_index']] += count_df['point_index']
        missing_points = num_points - gen_points
        keep_polygons = cupy.where(missing_points > 0)[0]
    # select exact number of points in each polygon
    selected_points = []
    count_points = keep_points.groupby('geoid').count()
    sorted_points = keep_points.sort_values(by='geoid').reset_index(drop=True)
    # start index of first point in this polygon
    point_id = 0
    for i in range(gdf.shape[0]):
        if i % 1000 == 0:
            print(f'Selecting exact number of points for the {i}-th polygon')
        num_expected_points = gdf[val_col][i]
        points = sorted_points.iloc[range(point_id, point_id + num_expected_points)]    
        geoid = geoids[i]
        num_generated_points = count_points['x'][geoid]
        point_id += num_generated_points
        selected_points.append(points)
    result = cudf.concat(selected_points, ignore_index=True)
    return result



As Census block data can be large, running them all at once can take long time we'll divide it into smaller batches to run it more efficiently.

In [ ]:
def get_subset(
    begin: int,                  # index of first row in the subset
    end: int,                    # index of last row in the subset
    gdf: gpd.GeoDataFrame,       # table of all polygons
    geoids: cupy.ndarray,        # indexes of all polygons
    poly_offsets: cudf.Series,   # polygon offsets
    ring_offsets: cudf.Series,   # ring offsets
    xcoords: cudf.Series,        # xcoords of polygons
    ycoords: cudf.Series,        # ycoords of polygons
):
    # select subset gdf
    subset_gdf = gdf.iloc[range(begin, end)]
    subset_geoids = geoids[begin:end]
    # index of the first ring in the subset
    begin_ring = poly_offsets[begin]
    # index of the last ring in the subset
    end_ring = poly_offsets[end]
    # select subset ring offsets
    subset_ring_offsets = ring_offsets.iloc[range(begin_ring, end_ring)].reset_index(drop=True)
    subset_ring_offsets = subset_ring_offsets - subset_ring_offsets[0]
    # subset poly offsets
    subset_poly_offsets = poly_offsets.iloc[range(begin, end)].reset_index(drop=True)  - poly_offsets[begin]
    # index of first coords
    begin_coords = ring_offsets[begin_ring]
    # index of last coords
    if end_ring < len(ring_offsets) - 1:
        end_coords = ring_offsets[end_ring]
    else:
        end_coords = len(coords)
    # select subset coords
    subset_xcoords = xcoords[begin_coords: end_coords].reset_index(drop=True)
    subset_ycoords = ycoords[begin_coords: end_coords].reset_index(drop=True)
    return (
        subset_gdf,
        subset_poly_offsets,
        subset_ring_offsets,
        subset_xcoords,
        subset_ycoords,
        subset_geoids
    )



## Experiments

We'll select run our dataset batch by batch with 2000 polygons each.

In [ ]:
val_col = 'POP'

batch_size = 2000
all_batch_results = []
for i in range(0, gdf.shape[0], batch_size):
    print(f'Processing the batch {i // batch_size + 1}')
    begin, end = i, i + batch_size
    subset_gdf, subset_poly_offsets, subset_ring_offsets, subset_xcoords, subset_ycoords, subset_geoids = get_subset(
        begin,
        end,
        gdf,
        geoids,
        poly_offsets,
        ring_offsets,
        xcoords,
        ycoords
    )
    subset_result = polygons_to_points(
        subset_gdf,
        val_col,
        subset_poly_offsets,
        subset_ring_offsets,
        subset_xcoords,
        subset_ycoords,
        subset_geoids,
        max_depth=10,
        min_size=500,
        max_len=3_000_000
    )
    all_batch_results.append(subset_result)

Combine all the results of all batches into a single dataframe.

In [ ]:
result = cudf.concat(all_batch_results, ignore_index=True)
result

**TODO**

- Select subset/batches of polygons by extent of `(xmin:xmax, ymin:ymax)` to avoid a batch where polygons are far away from some other, which leads to a big bounding box so this vectorization method becomes less efficient. (See batch 6: polygon 10000 to polygon 12000)

- Improve performance for selecting exact number of points in each polygon.

- Try cudf.read_parquet

- Move input to a public readable S3 bucket

- Save output to a S3 bucket

- Fine tune settings for quadtree: `max_depth`, `min_size`.

- Fine tune the number of random points `max_len` generated at each iteration.